<h1 style = "font-size: 30px; text-align: center;">AI Genetic Hands On</h1>
<h2 style = "font-size: 25px; text-align: center;">Hospital Job Scheduling</h2>
<h2 style = "font-size: 25px; text-align: center; color: #666">Name: Sepideh Fatemi</h2>
<h2 style = "font-size: 25px; text-align: center; color: #666">Student Id: 810896059</h2>
<h4 style="text-align: center">Spring 1400</h4>

<h2 style = "font-size: 25px;">Notes</h2>

<h3>Tests: </h3>

<p style="text-indent :2em;"><b>Test1: </b><mark>Average time < 15s</mark></p>
<p style="text-indent :2em;"><b>Test2: </b><mark>Average time < 60s</mark></p>
    

<h3>Outputs: </h3>
<p style="text-indent :2em;">Respectively <mark>output1.txt</mark> and <mark>output2.txt</mark></p>

<br>

In [1]:
%%html
<style>.text_cell .rendered_html * {direction: rtl; text-align: right;}</style>

In [2]:
import time
import random
from heapq import heappop, heappush, nlargest, nsmallest, heapify
from collections import Counter
import copy

هدف این پروژه استفاده از الگوریتم ژنتیک برای مسئله برنامه ریزی است.  
از این الگوریتم برای مسائل جست و جو و بهینه سازی استفاده می شود.

<h2 style = "font-size: 25px;">Test Files</h2>

In [3]:
testFile1 = "test1.txt"
testFile2 = "test2.txt"

<h2 style = "font-size: 25px;">Reading Test File Content</h2>

In [4]:
def readInput(testFile) :
    file = open(testFile, 'r+')
    fileList = file.readlines()
    fileList = [s.replace('\n', '') for s in fileList]
    
    [days, doctors] = [int(i) for i in fileList[0].split()]
    maxCapacity = int(fileList[1])
    
    allShifts = []
    for i in range(2, days + 2):
        dayRequirements = fileList[i].split()
        morningReqs = [int(i) for i in dayRequirements[0].split(",")]
        eveningReqs = [int(i) for i in dayRequirements[1].split(",")]
        nightReqs = [int(i) for i in dayRequirements[2].split(",")]
        
        allShifts.append((morningReqs, eveningReqs, nightReqs))

    file.close()
    return [days, list(range(doctors)), maxCapacity, allShifts]

<h2 style = "font-size: 25px;">Job Scheduler</h2>

In [5]:
class Chromosome:
    def __init__(self, genes, fitness):
        self.genes = genes
        self.fitness = fitness

    def __lt__(self, other):
        return self.fitness <= other.fitness

In [10]:
class JobScheduler:
    def __init__(self, fileInfo):
        self.days = fileInfo[0]
        self.doctors = len(fileInfo[1])
        self.doctorsIds = fileInfo[1]
        self.maxCapacity = fileInfo[2]
        self.allShifts = fileInfo[3]
        self.popSize = 500
        self.chromosomes = self.generateInitialPopulation()
        self.elitismPercentage = 0.6
        self.pc = 0.65
        self.pm = 0.8

    def generateInitialPopulation(self):
        population = []
        for i in range(self.popSize):
            chromosome_genes = []
            for d in range(self.days):
                day = {}
                for shift in range(3):
                    doctor_nom = random.randint(0, self.doctors)
                    day[shift] = set(random.sample(range(0, self.doctors), doctor_nom))
                chromosome_genes.append(day)
            fitness = self.calculateFitness(chromosome_genes)
            chromosome = Chromosome(chromosome_genes, fitness)
            heappush(population, chromosome)
        return population


    def crossOver(self, chromosome1, chromosome2):
        ch1_genes = copy.deepcopy(chromosome1.genes)
        ch2_genes = copy.deepcopy(chromosome2.genes)
        for day in range(self.days):
            for shift in range(3):
                if random.uniform(0, 1) < 0.5:
                    ch1_genes[day][shift], ch2_genes[day][shift] = ch2_genes[day][shift],  ch1_genes[day][shift]
        return ch1_genes


    def mutate(self, chromosome_genes):
        child_genes = copy.deepcopy(chromosome_genes)
        for day in range(self.days):
            for shift in range(3):
                if random.uniform(0, 1) < 0.5:
                    if not child_genes[day][shift]:
                        temp = random.sample(child_genes[day][shift], 1)[0]
                        child_genes[day][shift].remove(temp)
                if random.uniform(0, 1) < 0.5:
                    if not child_genes[day][shift]:
                        temp = random.sample(self.doctorsIds, 1)[0]
                        child_genes[day][shift].add(temp)
        return child_genes

    def calculateFitness(self, chromosome_genes):
        conflicts_no = 0
        dr_counts = Counter()
        for day in range(self.days):
            for shift in range(3):
                dr_counts.update(chromosome_genes[day][shift])
                if len(chromosome_genes[day][shift]) < self.allShifts[day][shift][0]:
                    conflicts_no += 1
                if len(chromosome_genes[day][shift]) > self.allShifts[day][shift][1]:
                    conflicts_no += 1

            if day + 1 < self.days and chromosome_genes[day][2] & chromosome_genes[day+1][0] and chromosome_genes[day][2] & chromosome_genes[day+1][1]:
                conflicts_no += 1
            if day + 2 < self.days and chromosome_genes[day][2] & chromosome_genes[day+1][2] & chromosome_genes[day+2][2]:
                conflicts_no += 3
        for dr in dr_counts:
            if dr_counts[dr] > self.maxCapacity:
                conflicts_no = conflicts_no + (dr_counts[dr] - self.maxCapacity)
        return -1/(conflicts_no+1)


    def generateNewPopulation(self):
        new_population = nsmallest(int(self.elitismPercentage * self.popSize), self.chromosomes)
        heapify(new_population)

        parents = copy.deepcopy(new_population)
        fitnesses = [ch.fitness for ch in self.chromosomes]
        probability_distribution = []
        total_fitness = sum(fitnesses)
        probability_distribution = [(ch.fitness/total_fitness) for ch in self.chromosomes]
        for i in range(int((1-self.elitismPercentage) * self.popSize)):
            parents.extend(random.choices(self.chromosomes, weights=probability_distribution))

        while len(new_population) < self.popSize:
            chromosome1 = random.choice(parents)
            chromosome2 = random.choice(parents)
            if self.pc > random.uniform(0, 1):
                child_genes = self.crossOver(chromosome1, chromosome2)
                child_fitness = self.calculateFitness(child_genes)
                child = Chromosome(child_genes, child_fitness)
                heappush(new_population, child)

        while len(new_population) < self.popSize:
            if self.pm > random.uniform(0, 1):
                chromosome = random.choice(parents)
                child_genes = self.mutate(chromosome.genes)
                child_fitness = self.calculateFitness(child_genes)
                child = Chromosome(child_genes, child_fitness)
                chr_dict = {new_population[i]: i for i in range(len(new_population))}
                index = chr_dict[chromosome]
                new_population[index] = new_population[-1]
                new_population.pop()
                heapify(new_population)
                heappush(new_population, child)

        max_fitness_chromosome = heappop(new_population)
        self.chromosomes = new_population
        return max_fitness_chromosome


    def schedule(self):
        while True:
            result = self.generateNewPopulation()
            if result.fitness == -1:
                print(result.genes)
                return result


def writeInFile(result, filename):
    a_file = open(filename, "w")
    for day in range(len(result.genes)):
        print(",".join(str(e) for e in result.genes[day][0]), end=' ', file=a_file)
        print(",".join(str(e) for e in result.genes[day][1]), end=' ', file=a_file)
        print(",".join(str(e) for e in result.genes[day][2]), file=a_file)
    a_file.close()


generateInitialPopulation:  
در این متد جمعیت اولیه را به صورت کاملا رندوم انتخاب می کنیم. برای هر شیفت به تعداد رندوم در هر شیفت دکتر رندومی را قرار می دهیم.
مدل سازی مسئله به این شکل است که هر کروموزوم لیستی است از روز ها و هر روز دیکشنری است از سه شیفت صبح و عصر و شب.  
کروموزوم روز ها و ژن ها شیفت های آن روز می باشد.  


در این پیاده سازی یک کلاس chromosome در نظر گرفتم که attribute های آن ژن های آن کروموزوم و fitness آن می باشد.  
دلیل این کار استفاده از heap برای کروموزوم می باشد که در این صورت با O(1) می توان کروموزوم با بیشترین فیتنس را در دست داشت . 

به دلیل این که heap در پایتون به صورت minheapاست و ما به دنبال بزرگ ترین fitness هستیم، مقادیر fitness را در -۱ ضرب کرده ام.  



crossOver:   
دو کروموزم می گیرد و به ازای هر شیفت به احتمال ۵۰ درصد از کروموزوم ۱ و به احتمال ۵۰ درصد از کروموزوم ۲ را نگه می دارد. 

mutate:  
     در این متد به ازای هر شیفت در کروموزوم بااحتمال ۵۰ درصد یا دکتری را از آن حذف می کند و یا اضافه می کند.  
     که ممکن است هردو یا هیچ کدام اتفاق بیافتد که در صورتی که هر دو اتفاق بیافتد به معنی عوض کردن یک دکتر با دکتر دیگر است.

calculateFitness:  
    به ازای هر محدودیت که در کروموزوم داده شده وجود دارد یکی به تعداد محدودیت ها اضافه میکنیم.  
به دلیل این که تعداد دکتر های اولیه بسیار زیاد به تعداد دکتر های اضافه، تعداد محدودیت را اضافه کردم.  
در نهایت فیتنس را معکوس تعداد محدودیت ها قرار دادم.   

generateNewPopulation:
با توجه به مقدار elitismPercentage درصدی از جمعیت اولیه را مستقیما به لیست parent ها اضافه می کنیم.  
تعداد parent ها باید برابر با تعداد جمعیت اولیه باشد.  
بقیه لیست parent را با استفاده از یک انتخاب رندوم وزن دار به وزن fitness/ total_fitness برای هر کروموزوم در نظر می گیریم.  
از لیست parent  ها دو کروموزوم را به صورت رندوم انتخاب می کنیم و آن ها را با احتمال pc کراس اور می کنیم و در جمعیت جدیدی که می خواهیم تشکیل دهیم قرار می دهیم.  
به همین ترتیب به صورت رندوم کروموزومی را از لیست parent ها انتخاب می کنیم و به احتمال pm  آ را mutate می کنیم.  
در نهایت جمعیت جدید را با جمعیت قبلی جایگزین می کنیم.


<h2 style = "font-size: 25px;">Execution</h2>

In [11]:
fileInfo1 = readInput(testFile1)
start = time.time()
scheduler = JobScheduler(fileInfo1)
result = scheduler.schedule()
end = time.time()
print("test 1: ", '%.2f' % (end - start), 'sec')
writeInFile(result, "output1.txt")

[{0: {1, 3, 5, 6}, 1: {4}, 2: {0, 1, 6}}, {0: {3, 4, 5}, 1: {4, 5}, 2: {4}}, {0: {6}, 1: {1, 4}, 2: {2}}, {0: {3}, 1: {0, 2}, 2: {3, 4, 6}}, {0: {2, 5}, 1: {5}, 2: {2}}, {0: {2}, 1: {0}, 2: {3}}, {0: {2, 3}, 1: {2}, 2: {0}}, {0: set(), 1: {1}, 2: {3, 5}}, {0: set(), 1: {6}, 2: {2}}, {0: {3}, 1: {4}, 2: {0, 5}}]
test 1:  5.83 sec


In [12]:
fileInfo2 = readInput(testFile2)
start = time.time()
scheduler = JobScheduler(fileInfo2)
result = scheduler.schedule()
end = time.time()
print("test 1: ", '%.2f' % (end - start), 'sec')
writeInFile(result, "output2.txt")

[{0: {0, 4}, 1: {1}, 2: {0, 1, 2, 4, 8}}, {0: {0, 3, 7}, 1: {6}, 2: {0}}, {0: {4}, 1: {3}, 2: {8, 2}}, {0: set(), 1: {1, 5}, 2: {8, 3, 5}}, {0: {1}, 1: {5}, 2: {4}}, {0: {8, 5}, 1: {3, 4, 6}, 2: {5, 7}}, {0: {3, 7}, 1: {6}, 2: {1, 4}}, {0: {5}, 1: {8, 5, 6}, 2: {2, 3}}, {0: {0}, 1: {0, 1, 2, 4}, 2: {8, 1, 3}}, {0: set(), 1: {0}, 2: set()}, {0: set(), 1: {4, 6}, 2: set()}, {0: {8, 1}, 1: {4}, 2: set()}, {0: {5}, 1: {4, 6}, 2: {2, 3}}, {0: {1}, 1: {2, 6}, 2: {8, 0, 6}}, {0: set(), 1: {2, 3}, 2: {1}}]
test 1:  11.26 sec


In [16]:
import os
os.system('jupyter nbconvert --to html genetic-hands-on.ipynb')

0